Sleep Quality Predicting Stress Levels

Loading Libraries

In [2]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

Warning message:
“package ‘ggplot2’ was built under R version 4.0.1”
Warning message:
“package ‘tibble’ was built under R version 4.0.2”
Warning message:
“package ‘tidyr’ was built under R version 4.0.2”
Warning message:
“package ‘dplyr’ was built under R version 4.0.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



Reading in data

In [4]:
data <- read_csv("data/SaYoPillow.csv")
data

Warning message:
“Duplicated column names deduplicated: 'sr' => 'sr_1' [7]”
Parsed with column specification:
cols(
  sr = col_double(),
  rr = col_double(),
  t = col_double(),
  lm = col_double(),
  bo = col_double(),
  rem = col_double(),
  sr_1 = col_double(),
  hr = col_double(),
  sl = col_double()
)



sr,rr,t,lm,bo,rem,sr_1,hr,sl
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
93.800,25.680,91.840,16.600,89.840,99.60,1.840,74.20,3
91.640,25.104,91.552,15.880,89.552,98.88,1.552,72.76,3
60.000,20.000,96.000,10.000,95.000,85.00,7.000,60.00,1
85.760,23.536,90.768,13.920,88.768,96.92,0.768,68.84,3
48.120,17.248,97.872,6.496,96.248,72.48,8.248,53.12,0
56.880,19.376,95.376,9.376,94.064,83.44,6.376,58.44,1
47.000,16.800,97.200,5.600,95.800,68.00,7.800,52.00,0
50.000,18.000,99.000,8.000,97.000,80.00,9.000,55.00,0
45.280,16.112,96.168,4.224,95.112,61.12,7.112,50.28,0


Cleaning and Wrangling

[insert units here]

In [10]:
clean_data <- rename(data, snoring_rate = sr, 
                     respiration_rate = rr, body_temperature = t, limb_movements = lm, 
                     blood_oxygen_levels = bo, eye_movements = rem, hours_sleep = sr_1, 
                     heart_rate = hr, stress_level = sl)
clean_data

snoring_rate,respiration_rate,body_temperature,limb_movements,blood_oxygen_levels,eye_movements,hours_sleep,heart_rate,stress_level
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
93.800,25.680,91.840,16.600,89.840,99.60,1.840,74.20,3
91.640,25.104,91.552,15.880,89.552,98.88,1.552,72.76,3
60.000,20.000,96.000,10.000,95.000,85.00,7.000,60.00,1
85.760,23.536,90.768,13.920,88.768,96.92,0.768,68.84,3
48.120,17.248,97.872,6.496,96.248,72.48,8.248,53.12,0
56.880,19.376,95.376,9.376,94.064,83.44,6.376,58.44,1
47.000,16.800,97.200,5.600,95.800,68.00,7.800,52.00,0
50.000,18.000,99.000,8.000,97.000,80.00,9.000,55.00,0
45.280,16.112,96.168,4.224,95.112,61.12,7.112,50.28,0
